In [1]:
import boto3
import json
import os
from functools import reduce
import pprint
from prettytable import PrettyTable
from youtube_transcript_api import YouTubeTranscriptApi
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.oauth2.credentials import Credentials

from phrasefinder import phrasefinder as pf
from sklearn.feature_extraction import stop_words
import time
from tqdm import tqdm
import statistics
from functools import reduce
from tinydb import TinyDB, Query
from google.cloud.bigquery_storage import BigQueryReadClient
from google.cloud.bigquery_storage import types
from google.cloud import bigquery
# from google.cloud import storage
import re
import os 
import sqlite3
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')


bigquery_client = None
bigquery_data = None
sql_conn = None

def getBestVideoList(keywords, topn=10):
    scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "0"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "./google_credentials/client_secret_754636752811-rmth1g8e3dl144jda8fddh1ihhj413um.apps.googleusercontent.com.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)

    credentials = Credentials(
        None,
        refresh_token="1//0fNppFYz3o7ABCgYIARAAGA8SNwF-L9IrgIZJAKCn9iSH_172SxyT6cA3mMHDlSQ0MTj9MmKTc6zZRnSy1nwMW5kRkl52JYb4jhg",
        token_uri="https://accounts.google.com/o/oauth2/token",
        client_id="754636752811-rmth1g8e3dl144jda8fddh1ihhj413um.apps.googleusercontent.com",
        client_secret="KhUufHmhS8XI0srgpP__cTCr")

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.search().list(
        part="snippet",
        maxResults=topn,
        q=keywords,
        relevanceLanguage='en'
    )
    response = request.execute()
    return response['items']


"""     best_content_list = []
    with os.scandir('./data') as entries:
        for entry in entries:
            with open('./data/' + entry.name, 'r') as file:
                best_content_list.append(file.read())
    return best_content_list """


def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # source_blob_name = "storage-object-name"
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Blob {} downloaded to {}.".format(
            source_blob_name, destination_file_name
        )
    )

def setupBiqqueryClient():
    print('Loading ngram data ...')
    global sql_conn
    sql_conn = sqlite3.connect('ngram.db')

    '''
    global bigquery_data
    bigquery_client = bigquery.Client()
    table_id = 'ngram_reddit_2019_08.ngram_reddit_2019_08'
    bigquery_data = {}
    
    NGRAM_FILE_NAME = 'ngram_2016_2019'
    if not os.path.exists(NGRAM_FILE_NAME):
        print('Downloading ngram from GCS ...')
        download_blob('ventureum-ngram', NGRAM_FILE_NAME, NGRAM_FILE_NAME)
        print('Download finished!')

    with open(NGRAM_FILE_NAME, 'r') as f:
        for line in f.readlines():
            data = json.loads(line)
            bigquery_data[data['phrase']] = {'phrase': data['phrase'], 'volume_count': int(data['volume_count']), 'match_count': int(data['match_count'])}
            '''

def getKeyPhrases(content_list):
    def insertFrequency(keyPhrase):
        frequency = {'overall': 0, 'min': 0,
                     'max': 0, 'frequency_by_content': []}
        for content in content_list:
            _freq = content.lower().count(keyPhrase['Text'].lower())
            frequency['overall'] += _freq
            frequency['min'] = _freq if frequency['min'] == 0 else min(
                frequency['min'], _freq)
            frequency['max'] = max(frequency['max'], _freq)
            frequency['frequency_by_content'].append(_freq)


        frequency['doc_freq'] = 0
        for e in frequency['frequency_by_content']:
            frequency['doc_freq'] += e > 0

        frequency['median'] = statistics.median(
            frequency['frequency_by_content'])

        frequency['doc_freq_percent'] = float(frequency['doc_freq']) / len(frequency['frequency_by_content'])
        return {**keyPhrase, 'frequency': frequency}


    def getNgramDataFromPhraseFinder(phrase):
        ngramData = {'text': phrase}

        # Optional: set the maximum number of phrases to return.
        options = pf.SearchOptions()
        options.topk = 1

        # Send the request.
        try:
            result = pf.search(pf.Corpus.AMERICAN_ENGLISH, phrase, options)
            if result.error:
                print('Request for {} was not successful: {}'.format(
                    phrase, result.error['message']))
                return

            time.sleep(0.02)
            ngramData = {
                **ngramData, 'volume_count': result.phrases[0].volume_count,
                'match_count': result.phrases[0].match_count}
        except Exception as error:
            print('Fatal error for {}: {}'.format(phrase, error))
            ngramData = {**ngramData, 'volume_count': 0, 'match_count': 0}
        
        return ngramData

    def getNgramDataFromBigQuery(phrase):
        global bigquery_data

        ngramData = {'text': phrase}
        '''
        if bigquery_data is None:
            setupBiqqueryClient()
        
        client = bigquery.Client()
        query_job = client.query(
            """
            SELECT *
            FROM `wallet-1546746037952.ngram_reddit_2019_08.ngram_reddit_2019_08`
            WHERE phrase = "{0}"
            """.format(phrase)
        )
        results = query_job.result()
        
        for row in results:
            ngramData = {**ngramData, 'volume_count': row.volume_count, 'match_count': row.match_count}
            return ngramData
        '''

        try:
            if sql_conn is None:
                setupBiqqueryClient()

            _str = phrase.lower()
            cur = sql_conn.cursor()
            query = "SELECT * FROM ngram_2016_2019 WHERE phrase = \"{}\"".format(_str)
            cur.execute(query)
            row = cur.fetchone()
            if row is None:
                print('Cannot find {}'.format(_str))
                ngramData = {**ngramData, 'volume_count': 0, 'match_count': 0}
            else:
                ngramData = {**ngramData, 'volume_count': row[2], 'match_count': row[1]}
        except sqlite3.OperationalError as e:
            print(e)
            ngramData = {**ngramData, 'volume_count': 0, 'match_count': 0}
        
        return ngramData
        

    def insertNgramFrequency(keyPhrase):

        db = TinyDB('db.json')
        ngramQuery = Query()

        # Set up your query.
        query = keyPhrase['Text']

        ngramData = db.get(ngramQuery.text == query)

        if ngramData is None:
            # ngramData = getNgramDataFromPhraseFinder(phrase)
            ngramData = getNgramDataFromBigQuery(query)
            # cache ngramData
            
            db.insert(ngramData)
        return {**keyPhrase, 'ngram_volume_count': ngramData['volume_count'], 'ngram_match_count': ngramData['match_count']}

    def filterPhrases(phrasesByContent):
        phrases = reduce(lambda x, y: x+y, phrasesByContent)
        # trim stop words from begining and end

        def stripStopWords(phrase):
            _stop_words = [
                "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
                "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
                "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had",
                "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as",
                "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through",
                "during", "before", "after", "above", "below", "to", "from", "in", "out", "on", "off",
                "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how",
                "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not",
                "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should",
                "now"] + ["yeah", "thing", "things", "hey", "ohh", "really", "mine", "everybody", "anybody", "everyone"]

            while True:
                # repetitively trim stop words from the beginning and the end
                modified = False
                _str = phrase['Text'].lower()
                for w in _stop_words:
                    if _str.startswith(w+' '):
                        phrase['Text'] = phrase['Text'][len(w)+1:]
                        modified = True
                        break
                    elif _str.endswith(' '+w):
                        phrase['Text'] = phrase['Text'][:-len(w)-1]
                        modified = True
                        break
                    elif _str == w:
                        phrase['Text'] = ""
                        break
                if not modified:
                    break
            return phrase

        
        phrases = list(map(lambda x: {**x, 'ori_text': x['Text']}, phrases))

        # substitue \n with whitespace
        phrases = list(map(lambda x: {**x, 'Text': x['Text'].replace("\n", "")}, phrases))

        phrases = list(map(stripStopWords, phrases))

        # only keep bigrams
        phrases = list(filter(lambda x: len(
            x['Text'].split(' ')) >= 1, phrases))

        # remove duplicates
        unique_phrases = []
        phrase_set = set()
        for phrase in phrases:
            if phrase['Text'].lower() not in phrase_set:
                phrase_set.add(phrase['Text'].lower())
                unique_phrases.append(phrase)

        print('# Unique phrases: {}'.format(len(unique_phrases)))

        # min keyword score
        unique_phrases = list(
            filter(lambda x: x['Score'] >= 0.99, unique_phrases))

        print('# phrases after min key word score: {}'.format(len(unique_phrases)))

         # phrase should have at least 3 chars
        unique_phrases = list(
            filter(lambda x: len(x['Text']) >= 3, unique_phrases))
        
        print('# phrases after min chars: {}'.format(len(unique_phrases)))

        # valid phrase should be words only, optionally connected by '-' or whitespace
        unique_phrases = list(filter(lambda phrase: bool(re.fullmatch(r'^[a-zA-Z0-9-\s]+$', phrase['Text'])), unique_phrases))

        print('# phrases after words only: {}'.format(len(unique_phrases)))

        # phrase should not contain only numbers, optionally connected by '-' or whitespace
        unique_phrases = list(filter(lambda phrase: not bool(re.fullmatch(r'^[\d\s-]+$', phrase['Text'])), unique_phrases))

        print('# phrases after not all numbers: {}'.format(len(unique_phrases)))

        # add frequency for each phrase
        unique_phrases = list(map(insertFrequency, unique_phrases))

        # keep phrases showing up at least once in 10% of documents
        unique_phrases = list(filter(lambda x: x['frequency']['doc_freq_percent'] >= 0.1, unique_phrases))

        print('# phrases after doc_freq_percent: {}'.format(len(unique_phrases)))

        # add ngrams frequency for each phrase
        for i in tqdm(range(len(unique_phrases))):
            unique_phrases[i] = insertNgramFrequency(unique_phrases[i])
        #unique_phrases = list(map(insertNgramFrequency, unique_phrases))

        phrases = unique_phrases

        # sort key phrases by overall frequency
        phrases.sort(reverse=True, key=lambda x: x['frequency']['overall'])

        print(len(phrases))

        # return most frequent top phrases
        return phrases

    # comprehend api requires each content to be less than 5000 bytes
    # see https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits.html
    content_list_splited = []
    for content in content_list:
        content_list_splited.extend([content[i: i+4000]
                                     for i in range(0, len(content), 4000)])
    """ for content in content_list_splited:
        print(len(content))
 """
    def chunks(l, n):
        for i in range(0, len(l), n):
            yield l[i:i + n]

    content_list_splited_chunks = list(chunks(content_list_splited, 25))

    keyPhrasesByContent = []
    entitiesByContent = []
    for chunk in content_list_splited_chunks:
        keyPhrasesResponse = comprehend.batch_detect_key_phrases(
            TextList=chunk, LanguageCode='en')
        entitiesResponse = comprehend.batch_detect_entities(
            TextList=chunk, LanguageCode='en')

        keyPhrasesByContent.extend(
            list(map(lambda x: x['KeyPhrases'], keyPhrasesResponse['ResultList'])))
        entitiesByContent.extend(
            list(map(lambda x: x['Entities'], entitiesResponse['ResultList'])))

    # return filterPhrases(keyPhrasesByContent), filterPhrases(entitiesByContent)
    return filterPhrases(keyPhrasesByContent)


def getTranscript(video_id):
    print('Getting transcript for '+video_id)
    return YouTubeTranscriptApi.get_transcript(video_id)


ModuleNotFoundError: No module named 'prettytable'

In [ ]:
# keywords = 'how do i get my annual free credit report'
keywords = 'How To Buy Your First Rental'
print('Working on keywords: ' + keywords)
video_list = getBestVideoList(keywords, 15)
# video_list = [{'id': {'videoId': 'kqMtDrsc5Pw'}}]
transcript_list = []
for video in video_list:
    try: 
        transcript_list.append(getTranscript(video['id']['videoId']))
    except:
        pass

content_list = []
for transcript in transcript_list:
    content = '. '.join(list(map(lambda x: x['text'], transcript)))
    # print(content)
    # print("\n\n")
    content_list.append(content)

In [ ]:
# setupBiqqueryClient()

In [ ]:
# keyPhrases, entities = getKeyPhrases(content_list)
keyPhrases = getKeyPhrases(content_list)

In [ ]:
import math

total_words = 0
def insertTfidf(phrase):
    # 1.4 M books
    # phrase['tfidf'] = (phrase['frequency']['overall'] / total_words) * (1+math.log(1180278/(1+phrase['ngram_volume_count'])))
    phrase['tfidf'] = math.log((phrase['frequency']['overall']+1)) * (1+math.log(34863800/(1+phrase['ngram_volume_count'])))
    # phrase['tfidf'] = (1) * (1+math.log(1180278/(1+phrase['ngram_volume_count'])))
    return phrase

def flatten(phrases):
    rv = []
    for phrase in phrases:
        rv.append({
            'phrase': phrase['Text'],
            'ori_text': phrase['ori_text'],
            'overall_freq': phrase['frequency']['overall'],
            'min_freq': phrase['frequency']['min'],
            'max_freq': phrase['frequency']['max'],
            'median_freq': phrase['frequency']['median'],
            'doc_freq_percent': phrase['frequency']['doc_freq_percent'],
            'ngram_volume_count': phrase['ngram_volume_count'],
            'ngram_match_count': phrase['ngram_match_count'],
            'tfidf': phrase['tfidf']
        })
    return rv

def printTable(phrases):
    table = PrettyTable()
    table.field_names=['phrase', 'ori_text', 'overall_freq', 'min_freq', 'max_freq', 'median_freq', 'doc_freq', 'ngram_volume_count', 'ngram_match_count', 'tfidf', 'entity_type']
    for phrase in phrases:
        entity_type = 'N/A'
        if 'Type' in phrase:
            entity_type = phrase['Type']
        table.add_row([phrase['Text'], phrase['ori_text'], phrase['Score'], phrase['frequency']['overall'], phrase['frequency']['min'], phrase['frequency']['max'], phrase['frequency']['median'], phrase['frequency']['doc_freq'], phrase['ngram_volume_count'], phrase['ngram_match_count'], phrase['tfidf'], entity_type])
    print(table)

for content in content_list:
    total_words += len(content.split())

print(len(keyPhrases))
keyPhrases = list(map(insertTfidf, keyPhrases))
#print top keyphrases table
keyPhrases.sort(reverse=True, key=lambda x: x['tfidf'])
flattenKeyPhrases = flatten(keyPhrases)
# printTable(keyPhrases)
#printTable(entities)
print(flattenKeyPhrases[5], flattenKeyPhrases[18])

In [ ]:
import matplotlib.pyplot as plt

def score(video_id, top_match=50):
    # get transcript of the video
    transcript = getTranscript(video_id)
    transcript_content = '. '.join(list(map(lambda x: x['text'], transcript)))

    report = {}
    matched = 0
    # check if keyword exist
    for keyPhrase in keyPhrases[:top_match]:
        _freq = transcript_content.lower().count(keyPhrase['Text'].lower())
        report[keyPhrase['Text']] = _freq
        matched += _freq > 0
    
    return matched*100.0/top_match, matched, report

# ranking >= 16

'''
video_list = ['Uwl3-jBNEd4', '1zd2Ev1OUzI', 'C-0tIsUCa_A', 'Y4jYNlDZvPw', 'h8R0qQDQebM', 'yB5qMIYh09Y', 'LxAniFgeCOg', 'KaMwnQlVoMI', '22pEyDoTkUk', 'a5RExfeU4UY', 'CPJp9Ch37UM', 'eH5lvuxvdCk', 'pjuRMKnVqhY', 'OXWssHtrteo', 'hdLPGmYHstg', '1i1KYJBPOzo', 'aGl3glZN74o', 'rFN_VOQjVqE', 'sM_DcT9ejdY', 'ghpYdSiLIyo', 'Ogre88cedwM', 'qs8TzflEkQg', '_HX22c0YD2s', 'UALlQsrbFas']
'''

# > 5m, 2k<=views<3k
'''
video_list = ['Zt7veuZb4_M', 'C-0tIsUCa_A', 'vMZSFGpAtcM', 'AY5jGiPiIBw', 'uiW8DHXpbNg', 'awLunjakH3Y', 'OBUl9ucfvGI', 'n6EFBt7BUPY', 'pCMsvfvnTXc', 'te9fn916zBY']
'''

# > 5m, 1k<=views<2k
'''
video_list = ['R_PB62qLrHE', '6sElkS_VjrI', 'EJ3haVDjgXY', '6X-SOg0-DDo', 'CZNvFLlDjgY', 'x7JosBVI9es', '4uXfwVt1wQU', 'tNuidxXRuDY', '2LucrGdfgFc', 'RXx73tPaoe8']
'''

# > 5m, 500<=views<1k
'''
video_list = ['UALlQsrbFas', 'zfkm53ehcQ0', 'IjPBzpkPFao', 'SR4FTQt9jYY', 'S_eMgsv5G1k', 'Z6sNAOCKEbM', 'SMqvz0rp22w', 'ZUDfRs3oLic', 'RcKxk6se4Us', 'LxL6bBTkGSo']
'''

# > 5m, 370<views<500
video_list = ['O76_DSyWbiU', 'KCLs7NmveIs', 'Euj3FyBisfA', 'P7_X25yr75w', 'ze3GLDE-c7U', 'BJXP4gt3gik', 'VwmGaiKkbiE', 'ElMgmKqtnTc', 'mjT7Lxe-lvU', '_GA-TMrG8RA']

print("testing video count: {}".format(len(video_list)))
chart_data = []
for video_id in video_list:
    try:
        match_perc, matched, report = score(video_id)
        print(match_perc, matched)
        chart_data.append(match_perc)
    except Exception as e:
        print('error scoring for video {} with err {}'.format(video_id, e))

print('median {}'.format(statistics.median(chart_data)))
print('mean {}'.format(statistics.mean(chart_data)))
plt.plot(chart_data)

